In [4]:
import numpy as np
import pandas as pd

import torch
import json
import datetime

from sklearn.model_selection import train_test_split

In [2]:
all_cards = json.load(open('/content/drive/MyDrive/Colab Notebooks/AllCards.json'))
all_sets = json.load(open('/content/drive/MyDrive/Colab Notebooks/SetList.json'))

In [3]:
print(len(all_cards))

20478


In [ ]:
cutoff_date = datetime.datetime(2003, 7, 21) # July 21, 2003
whitelisted_set_types = ['commander', 'core', 'draft_innovation', 'expansion']
legal_sets = list(filter(lambda set: datetime.datetime.strptime(set['releaseDate'], "%Y-%m-%d") > cutoff_date and set['type'] in whitelisted_set_types, all_sets))
legal_sets = [data['code'] for data in legal_sets]

illegal_sets = ['TSP']

In [ ]:
def filter_function(name_and_data):
  name, data = name_and_data
  return data['layout'] == 'normal' \
      and 'vintage' in data['legalities'].keys() \
      and any([printing in legal_sets for printing in data['printings']]) \
      and all([printing not in illegal_sets for printing in data['printings']]) \
      and all([banned_type not in data['types'] for banned_type in ['Land', 'Conspiracy']])

def select_relevant_fields_replace_name(data):
  fields = ['colors', 'convertedManaCost', 'name', 'power', 'toughness', 'text', 'type', 'types', 'supertypes', 'subtypes']
  selected = dict(filter(lambda kv: kv[0] in fields, data.items()))
  for field in fields:
    if field not in selected.keys():
      selected[field] = ''
  if selected['name'] in selected['text']:
    selected['text'] = selected['text'].replace(selected['name'], 'CARDNAME')
  try:
    selected['power'] = int(selected['power'])
  except:
    selected['power'] = 0
  try:
    selected['toughness'] = int(selected['toughness'])
  except:
    selected['toughness'] = 0

  return selected

In [ ]:
filtered_cards = dict(filter(filter_function, all_cards.items()))
filtered_cards = {name: select_relevant_fields_replace_name(data) for name, data in filtered_cards.items()}

In [ ]:
filtered_cards['Supreme Verdict']

{'colors': ['U', 'W'],
 'convertedManaCost': 4.0,
 'name': 'Supreme Verdict',
 'power': 0,
 'subtypes': [],
 'supertypes': [],
 'text': "This spell can't be countered.\nDestroy all creatures.",
 'toughness': 0,
 'type': 'Sorcery',
 'types': ['Sorcery']}

In [ ]:
print(f"Filtered {len(all_cards)} original cards to {len(filtered_cards)}")

Filtered 20478 original cards to 13106


In [ ]:
filtered_cards

{'Abandon Reason': {'colors': ['R'],
  'convertedManaCost': 3.0,
  'name': 'Abandon Reason',
  'power': 0,
  'subtypes': [],
  'supertypes': [],
  'text': 'Up to two target creatures each get +1/+0 and gain first strike until end of turn.\nMadness {1}{R} (If you discard this card, discard it into exile. When you do, cast it for its madness cost or put it into your graveyard.)',
  'toughness': 0,
  'type': 'Instant',
  'types': ['Instant']},
 'Abandoned Sarcophagus': {'colors': [],
  'convertedManaCost': 3.0,
  'name': 'Abandoned Sarcophagus',
  'power': 0,
  'subtypes': [],
  'supertypes': [],
  'text': "You may cast nonland cards with cycling from your graveyard.\nIf a card with cycling would be put into your graveyard from anywhere and it wasn't cycled, exile it instead.",
  'toughness': 0,
  'type': 'Artifact',
  'types': ['Artifact']},
 'Abattoir Ghoul': {'colors': ['B'],
  'convertedManaCost': 4.0,
  'name': 'Abattoir Ghoul',
  'power': 3,
  'subtypes': ['Zombie'],
  'supertypes':

In [ ]:
all_types = set()
all_subtypes = set()
all_supertypes = set()

for card_data in filtered_cards.values():
  all_types.update(card_data['types'])
  all_subtypes.update(card_data['subtypes'])
  all_supertypes.update(card_data['supertypes'])

In [ ]:
print('Total types:', len(all_types))
print('Total subtypes:', len(all_subtypes))
print('Total supertypes:', len(all_supertypes))

print('Types:', all_types)
print('Supertypes:', all_supertypes)

Total types: 7
Total subtypes: 270
Total supertypes: 2
Types: {'Planeswalker', 'Instant', 'Creature', 'Tribal', 'Sorcery', 'Enchantment', 'Artifact'}
Supertypes: {'Legendary', 'Snow'}


In [ ]:
card_list = list(filtered_cards.values())
print(len(card_list))

13106


In [ ]:
test_indices = np.random.choice(range(len(card_list)), int(0.2 * len(card_list)))

In [ ]:
train_data = []
test_data = []
for i, card in enumerate(card_list):
  if i in test_indices:
    test_data += [card]
  else:
    train_data += [card]
print(len(train_data), len(test_data))

10724 2382


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/mtg_train_data.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)
with open('/content/drive/MyDrive/Colab Notebooks/mtg_test_data.json', 'w', encoding='utf-8') as f:
    json.dump(test_data, f, ensure_ascii=False, indent=4)